In [82]:
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4

Things to think about:
* what if control day not in temperature data set? (solution is to add more temperature data)
* what if control day is actually a heat wave?
* what if control day is close to a heat wave?

In [83]:
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd



## Load heatwaves and get initial control dates

In [84]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/india/THI_india_90_2022.csv')

In [85]:
heatwaves.month.unique()

array([6, 7, 5, 4, 3])

In [86]:
#remove March bc COVID
#remove april so can search back
#remove = heatwaves.groupby('heat_ID').apply(lambda x: 3 in x['month'].unique()).reset_index()
#remove = remove[remove[0] == True]
#heatwaves = heatwaves[~heatwaves['heat_ID'].isin(remove['heat_ID'].values)]


#remove = heatwaves.groupby('heat_ID').apply(lambda x: 4 in x['month'].unique()).reset_index()
#remove = remove[remove[0] == True]
#heatwaves = heatwaves[~heatwaves['heat_ID'].isin(remove['heat_ID'].values)]

In [87]:
heatwaves['month'].unique()

array([6, 7, 5, 4, 3])

In [88]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [89]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)


In [90]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [91]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [92]:
heat_days = heat_days.to_dict()['datetime']

In [93]:
heatwaves_active.reset_index(inplace=True)

In [94]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/india/daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df.month.unique()
full_df = full_df[~full_df['month'].isin([8, 9])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]


In [95]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [96]:
full_df = full_df[~full_df['month'].isin([3])]
heatwaves_active = heatwaves_active[~heatwaves_active['month'].isin([3])]


In [97]:
heatwaves_active['num_iters'] = 'test'

In [98]:
heatwaves_active['month'].unique()

array([6, 7, 5, 4])

In [99]:
heatwaves_active.reset_index(drop=True, inplace=True)

In [100]:

for index, row in heatwaves_active.iterrows():
    
    i = 0
    try:
        while ((full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]>= 0.8) | 
               (full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]<= 0.2)):


                heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        
        while ((full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]>= 0.8) | 
               (full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            
            
    print(index / len(heatwaves_active))            
    heatwaves_active.iloc[index, -1] = i

0.0
0.00013042911177774878
0.00026085822355549756
0.0003912873353332464
0.0005217164471109951
0.000652145558888744
0.0007825746706664928
0.0009130037824442416
0.0010434328942219902
0.0011738620059997392
0.001304291117777488
0.0014347202295552368
0.0015651493413329856
0.0016955784531107344
0.0018260075648884831
0.001956436676666232
0.0020868657884439805
0.0022172949002217295
0.0023477240119994785
0.002478153123777227
0.002608582235554976
0.0027390113473327246
0.0028694404591104736
0.002999869570888222
0.003130298682665971
0.0032607277944437197
0.0033911569062214687
0.0035215860179992173
0.0036520151297769663
0.003782444241554715
0.003912873353332464
0.004043302465110213
0.004173731576887961
0.00430416068866571
0.004434589800443459
0.004565018912221208
0.004695448023998957
0.004825877135776705
0.004956306247554454
0.005086735359332203
0.005217164471109952
0.0053475935828877
0.005478022694665449
0.005608451806443198
0.005738880918220947
0.005869310029998695
0.005999739141776444
0.00613016

In [103]:
heatwaves_active['thi'].min()

7.9191129910197615

In [104]:
heatwaves['pct'].min()

0.9006535947712418

In [105]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves_active.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/india/THI_heatwave_control_90.csv')

In [106]:
df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/india/THI_heatwave_control_90.csv')

In [107]:
len(df['adm2'].unique())

715

In [108]:
pd.to_datetime(df['control_day']).dt.month.value_counts()

5    3665
6    2653
4     750
7     599
Name: control_day, dtype: int64